In [1]:
import time

### Load Data

In [2]:
# Parse the source
with open('dataset/THUOCL_chengyu.txt', 'r', encoding='utf-8') as source:
    idioms = [i for i in map(lambda x: x.split()[0], source.readlines()) if len(i) <= 4]

nodes = set([i[0] for i in idioms] + [i[-1] for i in idioms])
edges = [{"src": i[0], "dst": i[-1], "idiom": i} for i in idioms]

#### Create Graph

In [3]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", password="12345678")

cql = "CREATE \n" + ",".join("(%s : Node {char: \"%s\"})\n"%(n, n) for n in nodes)
for e in edges:
    cql += ",(%s)-[:idiom {idiom: '%s'}]->(%s)\n"%(e['src'], e["idiom"], e['dst'])

start = time.time()
graph.run("match (n) detach delete n")
graph.run(cql)
print("created graph, time elapsed: %.2fs"%(time.time() - start))

created graph, time elapsed: 8.89s


___
### Query Graph
##### Find next word

In [4]:
next_word = lambda word: graph.run("match ()-[:idiom {idiom: '%s'}]->()-[e:idiom]->() return e.idiom"%word)

df_next = next_word("坚定不移").to_data_frame()

print("found %i idioms"%len(df_next))
df_next.head()

found 3 idioms


,e.idiom
0,移花接木
1,移山填海
2,移风易俗


##### Find path

In [5]:
def find_path(idiom, length, limit):
    cql = "match ()-[e:idiom {idiom: '%s'}]->()"%idiom
    path = "".join(["-[e%i:idiom]->()"%i for i in range(length)])
    ret = ", ".join(["e%i.idiom"%i for i in range(length)])
    return graph.run(cql + path + " return e.idiom, " + ret + " limit %i"%limit).to_data_frame()

In [6]:
start = time.time()
df_paths = find_path(idiom="坚定不移", length=5, limit=100000)

print("found %i paths, time elapsed: %.2fs"%(len(df_paths), time.time() - start))
df_paths.head(10)

found 6833 paths, time elapsed: 0.30s


,e.idiom,e0.idiom,e1.idiom,e2.idiom,e3.idiom,e4.idiom
0,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心
1,坚定不移,移花接木,木牛流马,马到成功,功成身退,退避三舍
2,坚定不移,移花接木,木牛流马,马到成功,功德无量,量才录用
3,坚定不移,移花接木,木牛流马,马到成功,功德无量,量力而行
4,坚定不移,移花接木,木牛流马,马到成功,功德无量,量入为出
5,坚定不移,移花接木,木牛流马,马到成功,功德无量,量体裁衣
6,坚定不移,移花接木,木牛流马,马到成功,功德无量,量力而为
7,坚定不移,移花接木,木牛流马,马到成功,功臣自居,居心不良
8,坚定不移,移花接木,木牛流马,马到成功,功臣自居,居功厥伟
9,坚定不移,移花接木,木牛流马,马到成功,功臣自居,居无求安


In [7]:
start = time.time()
df_paths = find_path(idiom="坚定不移", length=10, limit=100000)

print("found %i paths, time elapsed: %.2fs"%(len(df_paths), time.time() - start))
df_paths.head(10)

found 100000 paths, time elapsed: 8.79s


,e.idiom,e0.idiom,e1.idiom,e2.idiom,e3.idiom,e4.idiom,e5.idiom,e6.idiom,e7.idiom,e8.idiom,e9.idiom
0,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒淫无道
1,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒无人烟
2,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒郊野外
3,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒淫无度
4,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒诞不经
5,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒谬绝伦
6,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒唐无稽
7,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒诞无稽
8,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地老天荒,荒淫无耻
9,坚定不移,移花接木,木牛流马,马到成功,功名富贵,贵在知心,心如刀割,割据一方,方寸之地,地利人和,和而不同
